<a href="https://colab.research.google.com/github/valid999/NLP_projects/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

dataset , info = tfds.load('imdb_reviews/subwords8k' , with_info= True ,
                           as_supervised = True)

train_dataset , test_dataset = dataset['train'] , dataset['test']

encoder = info.features['text'].encoder

BUFFER_SIZE =  10000
BATCH_SIZE  = 64


padded_shapes = ([None]  , ())

train_dataset = train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE , padded_shapes = padded_shapes)
test_dataset = test_dataset.padded_batch(BATCH_SIZE , padded_shapes = padded_shapes)


model = tf.keras.Sequential([tf.keras.layers.Embedding(encoder.vocab_size , 64) ,
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)) ,
                             tf.keras.layers.Dense(64 , activation = 'relu') ,
                             tf.keras.layers.Dense(1 , activation = 'sigmoid')
                             ])
model.compile(loss = 'binary_crossentropy' ,
              optimizer = tf.keras.optimizers.Adam(1e-4) ,
              metrics = ['accuracy'])

history = model.fit(train_dataset , epochs = 5 , validation_data = test_dataset , validation_steps = 30)


def pad_to_size(vec , size):
  zeros = [0] * (size-len(vec))
  vec.extend(zeros)
  return vec


# encoder goe is from higher dimensional representation to higher dimensional allows the make correlation between the words
def sample_predict(sentence , pad , model_):
  encoded_sample_pred_text = encoder.encode(sentence)
  if pad:
    encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text , 64)
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text , tf.float32)
  predictions = model_.predict(tf.expand_dims(encoded_sample_pred_text , 0))


  return predictions


sample_text = ('This movie was awesome .. The acting was incredible . Highly recommend')
predictions = sample_predict(sample_text , pad = True , model_ = model) * 100

print('Probability this is a positive review %.2f % predictions' % predictions)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteDA2RTX/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteDA2RTX/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteDA2RTX/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.
Epoch 1/5
391/391 [==============================] - 942s 2s/step - loss: 0.6605 - accuracy: 0.5873 - val_loss: 0.4934 - val_accuracy: 0.7729
Epoch 2/5
391/391 [==============================] - 920s 2s/step - loss: 0.3512 - accuracy: 0.8577 - val_loss: 0.3433 - val_accuracy: 0.8615
Epoch 3/5
391/391 [==============================] - 923s 2s/step - loss: 0.2531 - accuracy: 0.9074 - val_loss: 0.3355 - val_accuracy: 0.8745
Epoch 4/5
391/391 [==============================] - 933s 2s/step - loss: 0.2105 - accuracy: 0.9245 - val_loss: 0.3749 - val_accuracy: 0.8599
Epoch 5/5
1/1 [==============================] - 1s 835ms/step
Probability this is a positive review %.2f % predictions


In [ ]:
model = tf.keras.Sequential([tf.keras.layers.Embedding(encoder.vocab_size, 64),
            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(1, activation='sigmoid')])
model.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(1e-4),
                metrics=['accuracy'])

history = model.fit(train_dataset, epochs=5, validation_data=test_dataset,
                    validation_steps=30)
sample_text = ('This movie was awesome. The acting was incredible. Highly recommend')
predictions = sample_predict(sample_text, pad=True, model_=model) * 100

print('probability this is a positive review %.2f' % predictions)

sample_text = ('This movie was so so. The acting was medicore. Kind of recommend')
predictions = sample_predict(sample_text, pad=True, model_=model) * 100

print('probability this is a positive review %.2f' % predictions)